### Setup the connection 

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Myrto.Setzi/Documents/Ironhack/code_alongs/ironhack_service_account_big_query.json"

In [2]:
import six
six.moves.reload_module(six)

<module 'six' from 'C:\\Users\\myrto.setzi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\six.py'>

In [3]:
from google.cloud import bigquery

In [4]:
client = bigquery.Client()

### Challenge 1 - Most Profiting Authors?

### Step 1

In [5]:
query_1='''
SELECT 
    t.title_id
    , t.price
    , t.advance
    , t.royalty
    , s.qty
    , a.au_id
    , au_lname
    , au_fname
    , ta.royaltyper
    , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
FROM  
    `ironhack-data-analytics-265219.publications.titles` t
INNER JOIN 
    `ironhack-data-analytics-265219.publications.sales` s
ON
    s.title_id = t.title_id
INNER JOIN 
    `ironhack-data-analytics-265219.publications.titleauthor` ta
ON 
   ta.title_id = s.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.authors` a
ON 
    a.au_id = ta.au_id
ORDER BY
    t.title_id
    , a.au_id
'''

In [6]:
query_job_1 = client.query(query=query_1)

In [7]:
df_1=query_job_1.to_dataframe()

In [8]:
df_1.head(10)

,title_id,price,advance,royalty,qty,au_id,au_lname,au_fname,royaltyper,royalties
0,BU1032,19.99,5000.0,10,5,213-46-8915,Green,Marjorie,40,3.998
1,BU1032,19.99,5000.0,10,10,213-46-8915,Green,Marjorie,40,7.996
2,BU1032,19.99,5000.0,10,5,409-56-7008,Bennet,Abraham,60,5.997
3,BU1032,19.99,5000.0,10,10,409-56-7008,Bennet,Abraham,60,11.994
4,BU1111,11.95,5000.0,10,25,267-41-2394,O'Leary,Michael,40,11.950
5,BU1111,11.95,5000.0,10,25,724-80-9391,MacFeather,Stearns,60,17.925
6,BU2075,2.99,10125.0,24,35,213-46-8915,Green,Marjorie,100,25.116
7,BU7832,19.99,5000.0,10,15,274-80-9391,Straight,Dean,100,29.985
8,MC2222,19.99,0.0,12,10,712-45-1867,del Castillo,Innes,100,23.988
9,MC3021,2.99,15000.0,24,25,722-51-5454,DeFrance,Michel,75,13.455


### Step 2

In [9]:
query_2='''
SELECT
    title_id
    , au_id
    , au_lname
    , au_fname
    , advance
    , SUM(royalties) AS royalties
FROM
    (SELECT 
        t.title_id
        , t.price
        , t.advance
        , t.royalty
        , s.qty
        , a.au_id
        , au_lname
        , au_fname
        , ta.royaltyper
        , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
    FROM  
        `ironhack-data-analytics-265219.publications.titles` t
    INNER JOIN 
        `ironhack-data-analytics-265219.publications.sales` s
    ON
        s.title_id = t.title_id
    INNER JOIN 
        `ironhack-data-analytics-265219.publications.titleauthor` ta
    ON 
       ta.title_id = s.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.authors` a
    ON 
        a.au_id = ta.au_id
    ORDER BY
        t.title_id
        , a.au_id) roy
GROUP BY
    1,2,3,4,5
'''



In [10]:
query_job_2 = client.query(query=query_2)

In [11]:
df_2=query_job_2.to_dataframe()

In [12]:
df_2.head(10)

,title_id,au_id,au_lname,au_fname,advance,royalties
0,PS2091,899-46-2035,Ringer,Anne,2275.0,70.956
1,PS2091,998-72-3567,Ringer,Albert,2275.0,70.956
2,BU1032,213-46-8915,Green,Marjorie,5000.0,11.994
3,BU1032,409-56-7008,Bennet,Abraham,5000.0,17.991
4,PC8888,846-92-7186,Hunter,Sheryl,8000.0,50.000
5,PC8888,427-17-2319,Dull,Ann,8000.0,50.000
6,TC4203,648-92-1872,Blotchet-Halls,Reginald,4000.0,33.460
7,TC3218,807-91-6654,Panteley,Sylvia,7000.0,83.800
8,TC7777,267-41-2394,O'Leary,Michael,8000.0,8.994
9,TC7777,672-71-3249,Yokomoto,Akiko,8000.0,11.992


### Step 3

In [17]:
query_3='''
SELECT
    au_id AS author_id
    , au_lname AS last_name
    , au_fname AS first_name
    , SUM(advances + royalties) AS profits 
from 
    (SELECT
        title_id
        , au_id
        , au_lname
        , au_fname
        , SUM(advance) AS advances
        , SUM(royalties) AS royalties
    FROM
        (SELECT 
            t.title_id
            , t.price
            , t.royalty
            , s.qty
            , a.au_id
            , au_lname
            , au_fname
            , ta.royaltyper
            , (t.advance * ta.royaltyper / 100) AS advance
            , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
        FROM  
            `ironhack-data-analytics-265219.publications.titles` t
        INNER JOIN 
            `ironhack-data-analytics-265219.publications.sales` s
        ON
            s.title_id = t.title_id
        INNER JOIN 
            `ironhack-data-analytics-265219.publications.titleauthor` ta
        ON 
           ta.title_id = s.title_id
        INNER JOIN
            `ironhack-data-analytics-265219.publications.authors` a
        ON 
            a.au_id = ta.au_id
        ORDER BY
            t.title_id
            , a.au_id) roy
    GROUP BY
        1,2,3,4)prof
GROUP BY
    1,2,3
ORDER BY
    profits DESC
LIMIT 3
'''



In [18]:
query_job_3 = client.query(query=query_3)

In [19]:
df_3=query_job_3.to_dataframe()

In [20]:
df_3.head(10)

,author_id,last_name,first_name,profits
0,722-51-5454,DeFrance,Michel,22521.528
1,213-46-8915,Green,Marjorie,14162.110
2,899-46-2035,Ringer,Anne,12128.132


### Challenge 2 - Most Profiting Authors Table

In [ ]:
query_4='''
CREATE TABLE publications.most_profiting_authors AS
    au_id AS author_id
    , SUM(advances + royalties) AS profits 
from 
    (SELECT
        title_id
        , au_id
        , au_lname
        , au_fname
        , SUM(advance) AS advances
        , SUM(royalties) AS royalties
    FROM
        (SELECT 
            t.title_id
            , t.price
            , t.royalty
            , s.qty
            , a.au_id
            , au_lname
            , au_fname
            , ta.royaltyper
            , (t.advance * ta.royaltyper / 100) AS advance
            , (t.price * s.qty * t.royalty * ta.royaltyper / 10000) AS royalties
        FROM  
            `ironhack-data-analytics-265219.publications.titles` t
        INNER JOIN 
            `ironhack-data-analytics-265219.publications.sales` s
        ON
            s.title_id = t.title_id
        INNER JOIN 
            `ironhack-data-analytics-265219.publications.titleauthor` ta
        ON 
           ta.title_id = s.title_id
        INNER JOIN
            `ironhack-data-analytics-265219.publications.authors` a
        ON 
            a.au_id = ta.au_id
        ORDER BY
            t.title_id
            , a.au_id) roy
    GROUP BY
        1,2,3,4)prof
GROUP BY
    1,2,3
ORDER BY
    profits DESC
'''


In [ ]:
query_job_4 = client.query(query=query_4)